# Import packages and database

In [170]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.segmentation as seg
from biogeme.expressions import Beta, Variable, Derive, exp
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
from collections import namedtuple

data_file ='https://raw.githubusercontent.com/GustavePellier/MMOB/main/lpmc19.dat'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC

database = db.Database('LPMC', LPMC)
all_results = {}

# Model 0

We calculate the total public transport duration and the total driving cost

In [171]:
LPMC["dur_pt"]= LPMC["dur_pt_access"] + LPMC["dur_pt_rail"] + LPMC["dur_pt_bus"] + LPMC["dur_pt_int"] 
LPMC["cost_drive"] = LPMC["cost_driving_ccharge"] + LPMC["cost_driving_fuel"]

Some variables are created with the columns that seem to be useful

In [172]:
travel_mode=Variable('travel_mode')

dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')

pt_interchanges=Variable('pt_interchanges')

cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
cost_drive=Variable('cost_drive')
cost_pt=Variable('cost_transit')

time_walk=Variable('dur_walking')
time_cycle=Variable('dur_cycling')
time_pt=Variable('dur_pt')
time_drive=Variable('dur_driving')

female=Variable('female')
age=Variable('age')

There are 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [173]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

Utility functions creation

In [174]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

availability of each mode, all available here

In [175]:
av = {1: 1, 2: 1, 3: 1, 4:1}

The estimation results (parameter values, t-tests or p-values, null and final log likelihoods)

In [176]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.878620,0.107672,-36.022417,0.0
asc_drive,-1.295192,0.080505,-16.088357,0.0
asc_pt,-0.503354,0.054123,-9.300121,0.0
beta_cost,-0.193629,0.013958,-13.871886,0.0
beta_time,-5.495530,0.208596,-26.345334,0.0


In [177]:
res.compile_estimation_results(all_results)

(                                         Model0
 Number of estimated parameters                5
 Sample size                                5000
 Final log likelihood               -4587.818071
 Akaike Information Criterion        9185.636142
 Bayesian Information Criterion      9218.222108
 asc_cycle (t-test)                 -3.88  (-36)
 asc_drive (t-test)                -1.3  (-16.1)
 asc_pt (t-test)                  -0.503  (-9.3)
 beta_cost (t-test)              -0.194  (-13.9)
 beta_time (t-test)                -5.5  (-26.3),
 {'Model0': 'Model0'})

In [178]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1

We choose to do a specification for the time attribute, the cost stays generic in this model. Therefore we obtain a beta_time parameter for each transportation mode. 

In [179]:
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)

The utility functions are rewritten with the new parameters :

In [180]:
v_walk_model1= beta_time_walk * time_walk  
v_cycle_model1= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1= asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1= asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

The results are computed (parameter values, t-tests or p-values, null and final log likelihoods)

In [181]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model_1'
all_results['Model1'] = biogeme_model1.estimate()
results_specific_time = biogeme_model1.estimate()
results_specific_time.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.847224,0.202570,-23.928655,0.0
asc_drive,-1.969874,0.137929,-14.281807,0.0
asc_pt,-2.337196,0.139506,-16.753340,0.0
beta_cost,-0.182009,0.016087,-11.314048,0.0
beta_time_cycle,-5.312452,0.462002,-11.498765,0.0
beta_time_drive,-6.474307,0.378989,-17.083106,0.0
beta_time_pt,-3.538619,0.250656,-14.117425,0.0
beta_time_walk,-8.430768,0.418991,-20.121615,0.0


In [182]:
res.compile_estimation_results(all_results)

(                                         Model0           Model1
 Number of estimated parameters                5                8
 Sample size                                5000             5000
 Final log likelihood               -4587.818071     -4275.496542
 Akaike Information Criterion        9185.636142      8566.993084
 Bayesian Information Criterion      9218.222108       8619.13063
 asc_cycle (t-test)                 -3.88  (-36)   -4.85  (-23.9)
 asc_drive (t-test)                -1.3  (-16.1)   -1.97  (-14.3)
 asc_pt (t-test)                  -0.503  (-9.3)   -2.34  (-16.8)
 beta_cost (t-test)              -0.194  (-13.9)  -0.182  (-11.3)
 beta_time (t-test)                -5.5  (-26.3)                 
 beta_time_cycle (t-test)                          -5.31  (-11.5)
 beta_time_drive (t-test)                          -6.47  (-17.1)
 beta_time_pt (t-test)                             -3.54  (-14.1)
 beta_time_walk (t-test)                           -8.43  (-20.1),
 {'Model0

In [183]:
biogeme_model1.calculateNullLoglikelihood(av)

-6931.471805599917

We compare the 2 models with the likelihood ratio test because model 0 is a reduction of model 1.  

In [184]:
results_specific_time.likelihood_ratio_test(results_generic, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=624.6430583536785, threshold=7.814727903251179)

# Model 2

Using Model 1 as the base model, we choose to study how gender interact with the
ASCs and the alternative attributes. 

Lets start by segmenting the population by gender

In [185]:
gender_segmentation = seg.DiscreteSegmentationTuple(variable=female, mapping={1: 'female', 0: 'male'})

### ASC segmentation

In [186]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
segmented_asc_cycle = seg.Segmentation(asc_cycle,[gender_segmentation]).segmented_beta()

asc_pt = Beta('asc_pt', 0, None, None, 0)
segmented_asc_pt = seg.Segmentation(asc_pt,[gender_segmentation]).segmented_beta()

asc_drive = Beta('asc_drive', 0, None, None, 0)
segmented_asc_drive = seg.Segmentation(asc_drive,[gender_segmentation]).segmented_beta()

We redefine the value function of the 4 alternatives, introducing the segmented ASC and the segmented cost attribute

In [187]:
v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

We can compute the result of the new model

In [188]:
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model2'] = biogeme_model2.estimate()
results_segmented_gender = biogeme_model2.estimate()
results_segmented_gender.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-5.454906,0.253359,-21.530353,0.000000e+00
asc_cycle_male,1.051966,0.208777,5.038696,4.687141e-07
asc_drive,-1.893648,0.144299,-13.123111,0.000000e+00
asc_drive_male,-0.197931,0.095253,-2.077955,3.771347e-02
asc_pt,-2.208481,0.146369,-15.088464,0.000000e+00
asc_pt_male,-0.333133,0.099553,-3.346279,8.190405e-04
beta_cost,-0.182829,0.016154,-11.317962,0.000000e+00
beta_time_cycle,-5.580986,0.473158,-11.795183,0.000000e+00
beta_time_drive,-6.487796,0.380048,-17.070999,0.000000e+00
beta_time_pt,-3.526902,0.251400,-14.029021,0.000000e+00


In [189]:
biogeme_model2.calculateNullLoglikelihood(av)

-6931.471805599917

### Cost segmentation

And its interaction on the cost attribute in the car and public transport alternative.

In [190]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
segmented_beta_cost = seg.Segmentation(beta_cost, [gender_segmentation]).segmented_beta()

In [191]:
v_walk_model2_2= beta_time_walk * time_walk  
v_cycle_model2_2= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2_2= asc_pt + beta_time_pt * time_pt + segmented_beta_cost * cost_pt
v_drive_model2_2= asc_drive + beta_time_drive * time_drive + segmented_beta_cost * cost_drive

We can compute the result of the new model

In [192]:
V_model2_2 = {1: v_walk_model2_2 , 2: v_cycle_model2_2, 3: v_pt_model2_2, 4: v_drive_model2_2}
logprob_model2_2 = models.loglogit(V_model2_2, av, travel_mode)
biogeme_model2_2 = bio.BIOGEME(database, logprob_model2_2)
biogeme_model2_2.modelName = 'Model_2_2'
all_results['Model2_2'] = biogeme_model2_2.estimate()
results_segmented_gender_2 = biogeme_model2_2.estimate()
results_segmented_gender_2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.843793,0.202194,-23.956159,0.000000e+00
asc_drive,-1.971743,0.137733,-14.315669,0.000000e+00
asc_pt,-2.343140,0.139310,-16.819666,0.000000e+00
beta_cost,-0.151591,0.018749,-8.085476,6.661338e-16
beta_cost_male,-0.065394,0.031840,-2.053794,3.999561e-02
beta_time_cycle,-5.348638,0.459248,-11.646508,0.000000e+00
beta_time_drive,-6.483024,0.380414,-17.042046,0.000000e+00
beta_time_pt,-3.536891,0.250704,-14.107825,0.000000e+00
beta_time_walk,-8.435703,0.418411,-20.161281,0.000000e+00


In [193]:
biogeme_model2_2.calculateNullLoglikelihood(av)

-6931.471805599917

### Compiling all results from the different models

In [194]:
res.compile_estimation_results(all_results)

(                                         Model0           Model1  \
 Number of estimated parameters                5                8   
 Sample size                                5000             5000   
 Final log likelihood               -4587.818071     -4275.496542   
 Akaike Information Criterion        9185.636142      8566.993084   
 Bayesian Information Criterion      9218.222108       8619.13063   
 asc_cycle (t-test)                 -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                  -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)              -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                -5.5  (-26.3)                    
 beta_time_cycle (t-test)                          -5.31  (-11.5)   
 beta_time_drive (t-test)                          -6.47  (-17.1)   
 beta_time_pt (t-test)                             -3.54  (-14.1)   
 beta_time_walk (t-test)          

### Likelihood ratio test to check if model_2 and model_2_2 are equivalent or not

In [195]:
results_segmented_gender.likelihood_ratio_test(results_segmented_gender_2, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=55.92757903317943, threshold=5.991464547107979)

Model_2 is preferred because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2

### Comparison with model 1

In [196]:
results_segmented_gender.likelihood_ratio_test(results_specific_time, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=60.51262181686252, threshold=7.814727903251179)

Model_2 is preferred to Model_1 because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2.

# Model 3

In [197]:
# Repartir du model segmenté, Modèle 2

In [198]:
#GENDER segmentation
gender_segmentation_3bis = seg.DiscreteSegmentationTuple(variable=female, mapping={1: 'female', 0: 'male'})

In [199]:
asc_cycle_3bis = Beta('asc_cycle_3bis', 0, None, None, 0)
segmented_asc_cycle_3bis = seg.Segmentation(asc_cycle,[gender_segmentation]).segmented_beta()

asc_pt_3bis = Beta('asc_pt_3bis', 0, None, None, 0)
segmented_asc_pt_3bis = seg.Segmentation(asc_pt,[gender_segmentation]).segmented_beta()

asc_drive_3bis = Beta('asc_drive_3bis', 0, None, None, 0)
segmented_asc_drive_3bis = seg.Segmentation(asc_drive_3bis,[gender_segmentation_3bis]).segmented_beta()

In [200]:
beta_time_walk_3bis = Beta('beta_time_walk_3bis', 0, None, None, 0)
beta_time_cycle_3bis = Beta('beta_time_cycle_3bis', 0, None, None, 0)

beta_time_pt_3bis = Beta('beta_time_pt_3bis', 0, None, None, 0)
beta_time_pt_3bis_squarred = Beta('beta_time_pt_3bis_squarred', 0, None, None, 0)

beta_time_drive_3bis = Beta('beta_time_drive_3bis', 0, None, None, 0)
beta_cost_3bis = Beta('beta_cost_3bis', 0, None, None, 0)


v_walk_model_3bis= beta_time_walk_3bis * time_walk  

v_cycle_model_3bis= segmented_asc_cycle_3bis + beta_time_cycle_3bis * time_cycle 

v_pt_model_3bis= segmented_asc_pt_3bis + beta_time_pt_3bis * time_pt + beta_time_pt_3bis_squarred *time_pt*time_pt + beta_cost_3bis * cost_pt

v_drive_model_3bis= segmented_asc_drive_3bis + beta_time_drive_3bis * time_drive + beta_cost_3bis * cost_drive

In [201]:
V_model_3bis = {1: v_walk_model_3bis , 2: v_cycle_model_3bis, 3: v_pt_model_3bis, 4: v_drive_model_3bis}
logprob_model_3bis = models.loglogit(V_model_3bis, av, travel_mode)
biogeme_model_3bis = bio.BIOGEME(database, logprob_model_3bis)
biogeme_model_3bis.modelName = 'Model_3bis'
all_results['Model_3bis'] = biogeme_model_3bis.estimate()
results_segmented_gender_3bis = biogeme_model_3bis.estimate()
results_segmented_gender_3bis.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-5.373633,0.250689,-21.435415,0.000000e+00
asc_cycle_male,1.052224,0.208454,5.047748,4.470478e-07
asc_drive_3bis,-1.802140,0.145490,-12.386676,0.000000e+00
asc_drive_3bis_male,-0.198042,0.094447,-2.096845,3.600732e-02
asc_pt,-2.490305,0.160124,-15.552319,0.000000e+00
asc_pt_male,-0.339756,0.101136,-3.359391,7.811443e-04
beta_cost_3bis,-0.180475,0.016392,-11.010049,0.000000e+00
beta_time_cycle_3bis,-5.573343,0.458404,-12.158134,0.000000e+00
beta_time_drive_3bis,-6.562530,0.388238,-16.903348,0.000000e+00
beta_time_pt_3bis,-1.983146,0.470332,-4.216483,2.481418e-05


In [202]:
#COST segmentation

In [203]:
beta_cost_33 = Beta('beta_cost_33', 0, None, None, 0)
segmented_beta_cost_33 = seg.Segmentation(beta_cost_33, [gender_segmentation_3bis]).segmented_beta()

In [204]:
beta_time_pt_33_squarred = Beta('beta_time_pt_33_squarred', 0, None, None, 0)

v_walk_model_33= beta_time_walk * time_walk  

v_cycle_model_33= asc_cycle + beta_time_cycle * time_cycle 

v_pt_model_33= asc_pt + beta_time_pt * time_pt + beta_time_pt_33_squarred * time_pt * time_pt + segmented_beta_cost * cost_pt

v_drive_model_33= asc_drive + beta_time_drive * time_drive + segmented_beta_cost * cost_drive

In [205]:
V_model_33 = {1: v_walk_model_33 , 2: v_cycle_model_33, 3: v_pt_model_33, 4: v_drive_model_33}
logprob_model_33 = models.loglogit(V_model_33, av, travel_mode)
biogeme_model_33 = bio.BIOGEME(database, logprob_model_33)
biogeme_model_33.modelName = 'Model__33'
all_results['Model_33'] = biogeme_model_33.estimate()
results_segmented_gender_33 = biogeme_model_33.estimate()
results_segmented_gender_33.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.761737,0.199493,-23.869146,0.000000e+00
asc_drive,-1.881527,0.139072,-13.529112,0.000000e+00
asc_pt,-2.623410,0.154065,-17.027935,0.000000e+00
beta_cost,-0.149036,0.018925,-7.875277,3.330669e-15
beta_cost_male,-0.065963,0.032234,-2.046418,4.071531e-02
beta_time_cycle,-5.345713,0.445822,-11.990691,0.000000e+00
beta_time_drive,-6.555686,0.388305,-16.882845,0.000000e+00
beta_time_pt,-2.013847,0.470794,-4.277554,1.889584e-05
beta_time_pt_33_squarred,-1.219341,0.352244,-3.461636,5.369018e-04
beta_time_walk,-8.288616,0.418773,-19.792639,0.000000e+00


### Comparison model_3bis and model_33

In [206]:


results_segmented_gender_3bis.likelihood_ratio_test(results_segmented_gender_33, 0.05)


LRTuple(message='H0 can be rejected at level 5.0%', statistic=56.37685201186832, threshold=5.991464547107979)

In [207]:
print(results_segmented_gender_3bis.printGeneralStatistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4236.589
Final log likelihood:	-4236.589
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00283
Akaike Information Criterion:	8497.178
Bayesian Information Criterion:	8575.385
Final gradient norm:	2.8382E-04
Nbr of threads:	8



In [208]:
print(results_segmented_gender_33.printGeneralStatistics())

Number of estimated parameters:	10
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4264.778
Final log likelihood:	-4264.778
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00234
Akaike Information Criterion:	8549.555
Bayesian Information Criterion:	8614.727
Final gradient norm:	1.9340E-04
Nbr of threads:	8



### Preferred model is model_3bis

The model_3bis is preferred to model_33 as its log likelihood is greater and the AIC and BIC are smaller as well for this model.

### Comparison model_3bis and model_2

In [209]:
results_segmented_gender_3bis.likelihood_ratio_test(results_segmented_gender, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=17.302101677916653, threshold=3.841458820694124)

In [210]:
print(results_segmented_gender.printGeneralStatistics())

Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4245.24
Final log likelihood:	-4245.24
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00259
Akaike Information Criterion:	8512.48
Bayesian Information Criterion:	8584.17
Final gradient norm:	1.9456E-03
Nbr of threads:	8



We notice that the Final loglikelyhood as well as AIC & BIC are lower for the model_3bis than for model_2.
As those two model are not equivalent at the 5% significance level, we can conclude that the model 3 bis is preferred

### Version ancienne du power series 

In [211]:
# Power Series transformation on 'pt_dur'.

asc_cycle_3 = Beta('asc_cycle_3', 0, None, None, 0)

asc_pt_3 = Beta('asc_pt_3', 0, None, None, 0)

asc_drive_3 = Beta('asc_drive_3', 0, None, None, 0)


beta_time_walk_3 = Beta('beta_time_walk_3', 0, None, None, 0)
beta_time_cycle_3 = Beta('beta_time_cycle_3', 0, None, None, 0)

beta_time_pt_3 = Beta('beta_time_pt_3', 0, None, None, 0)
beta_time_pt_3_squarred = Beta('beta_time_pt_3_squarred', 0, None, None, 0)

beta_time_drive_3 = Beta('beta_time_drive_3', 0, None, None, 0)
beta_cost_3 = Beta('beta_cost_3', 0, None, None, 0)



v_walk_model_3= beta_time_walk_3 * time_walk 

v_cycle_model_3= asc_cycle_3 + beta_time_cycle_3 * time_cycle 

v_pt_model_3= asc_pt_3 + beta_time_pt_3 * time_pt +beta_time_pt_3_squarred * time_pt * time_pt + beta_cost * cost_pt

v_drive_model_3= asc_drive_3 + beta_time_drive_3 * time_drive + beta_cost_3 * cost_drive

In [212]:
V_model_3 = {1: v_walk_model_3 , 2: v_cycle_model_3, 3: v_pt_model_3, 4: v_drive_model_3}

logprob_model_3 = models.loglogit(V_model_3, av, travel_mode)

biogeme_model_3 = bio.BIOGEME(database, logprob_model_3)

biogeme_model_3.modelName = 'Model_3'

all_results['Model_3'] = biogeme_model_3.estimate()

results_power_3 = biogeme_model_3.estimate()

results_power_3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle_3,-4.763434,0.199847,-23.835399,0.000000e+00
asc_drive_3,-1.873748,0.139320,-13.449255,0.000000e+00
asc_pt_3,-2.601264,0.155172,-16.763701,0.000000e+00
beta_cost,-0.210770,0.031140,-6.768429,1.301892e-11
beta_cost_3,-0.168058,0.017600,-9.548949,0.000000e+00
beta_time_cycle_3,-5.431875,0.464148,-11.702905,0.000000e+00
beta_time_drive_3,-6.753131,0.442152,-15.273340,0.000000e+00
beta_time_pt_3,-2.024757,0.471819,-4.291382,1.775648e-05
beta_time_pt_3_squarred,-1.228551,0.352701,-3.483264,4.953405e-04
beta_time_walk_3,-8.319325,0.421317,-19.746000,0.000000e+00


In [213]:
#Calculate Null Log likelihood

biogeme_model_3.calculateNullLoglikelihood(av)

-6931.471805599917

In [214]:
#Comparison to model 1

results_power_3.likelihood_ratio_test(results_specific_time, 0.05)




LRTuple(message='H0 can be rejected at level 5.0%', statistic=18.42252239267509, threshold=5.991464547107979)

The model 3 is not preferred compare to model 1.

-Is it due to the non linear transformation that is not appropriate ? Should we use boxplot instead ? --> Yes we should 

-Is it stated 'non-linear transfronation on one of the variables' so I just took public transport time, should I apply to the other time ? --> I think but I will ask

-Should the non linear transformation be applied to an other time variable like car ? Yes I think but I will ask

-Is the loglikelyhood ratio test still appropirate ? --> Maybe we should use a cox-text instead 

Remarks: I think that the non-linear transformation is not done on beta but rather on variables such as time, cost, etc.!


# Model 3 - Salomé

In [215]:

lambda_boxcox = Beta('lambda_boxcox', 1, None, None, 0) #vu dans le modèle 01-logit_airline_solution
boxcox_time_walk = models.boxcox(time_walk, lambda_boxcox)
boxcox_time_cycle = models.boxcox(time_cycle, lambda_boxcox)
boxcox_time_pt = models.boxcox(time_pt, lambda_boxcox)
boxcox_time_drive = models.boxcox(time_drive , lambda_boxcox)
beta_elapsed_time_walk = Beta('beta_elapsed_walk', 0, None, None, 0)
beta_elapsed_time_cycle = Beta('beta_elapsed_cycle', 0, None, None, 0)
beta_elapsed_time_pt = Beta('beta_elapsed_time_pt', 0, None, None, 0)
beta_elapsed_time_drive = Beta('beta_elapsed_time_drive', 0, None, None, 0)


v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive


v_walk_model3 = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model3  = segmented_asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model3  = segmented_asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt 
v_drive_model3  = segmented_asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive 

It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)
It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)
It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)
It is advised to set the bounds on parameter lambda_boxcox. A value of -10 and 10 should be appropriate: Beta("lambda_boxcox", 1, -10, 10, 0)


In [216]:
V_model3 = {1: v_walk_model3 , 2: v_cycle_model3, 3: v_pt_model3, 4: v_drive_model3}
logprob_model3 = models.loglogit(V_model3, av, travel_mode)
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.modelName = 'Model_3'
all_results['Model_3'] = biogeme_model3.estimate()
results_model3 = biogeme_model3.estimate()
results_model3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.218625,0.303923,-10.590262,0.000000e+00
asc_cycle_male,1.053238,0.210050,5.014213,5.325099e-07
asc_drive,0.068987,0.217651,0.316961,7.512731e-01
asc_drive_male,-0.226069,0.101484,-2.227638,2.590466e-02
asc_pt,1.844828,0.170761,10.803571,0.000000e+00
asc_pt_male,-0.375192,0.107310,-3.496329,4.717070e-04
beta_cost,-0.174891,0.015669,-11.161365,0.000000e+00
beta_elapsed_cycle,-3.318145,0.268400,-12.362686,0.000000e+00
beta_elapsed_time_drive,-3.301154,0.265806,-12.419408,0.000000e+00
beta_elapsed_time_pt,-2.637900,0.182666,-14.441116,0.000000e+00


In [217]:
res.compileEstimationResults(all_results)

The syntax "compileEstimationResults" is deprecated and is replaced by the syntax "compile_estimation_results".


(                                              Model0           Model1  \
 Number of estimated parameters                     5                8   
 Sample size                                     5000             5000   
 Final log likelihood                    -4587.818071     -4275.496542   
 Akaike Information Criterion             9185.636142      8566.993084   
 Bayesian Information Criterion           9218.222108       8619.13063   
 asc_cycle (t-test)                      -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                     -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                       -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)                   -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                     -5.5  (-26.3)                    
 beta_time_cycle (t-test)                               -5.31  (-11.5)   
 beta_time_drive (t-test)                               -6.47  (-17.1)   
 beta_time_pt (t-test)                

In [218]:
print("Null Loglikelihood : ")
biogeme_model3.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

In [219]:
results_model3.likelihood_ratio_test(results_segmented_gender, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=175.987729750801, threshold=3.841458820694124)

In [220]:
print(results_model3.printGeneralStatistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4157.246
Final log likelihood:	-4157.246
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00289
Akaike Information Criterion:	8338.493
Bayesian Information Criterion:	8416.699
Final gradient norm:	9.3813E-04
Nbr of threads:	8



To compare Model 2 and Model 3 I think we should use a Cox test as Model 3 is not a a linear restriction of Model 2, to do that we create a composite model C such that both models 1 and 2 are restricted cases of model C.


In [221]:
v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive


v_walk_model3 = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model3  = segmented_asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model3  = segmented_asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt 
v_drive_model3  = segmented_asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive 


v_walk_model_generic = beta_time_walk * time_walk + beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model_generic = segmented_asc_cycle + beta_time_cycle * time_cycle +  beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model_generic = segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt + beta_elapsed_time_pt * boxcox_time_pt 
v_drive_model_generic = segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive + beta_elapsed_time_drive * boxcox_time_drive

In [222]:
V_model_generic = {1: v_walk_model_generic , 2: v_cycle_model_generic, 3: v_pt_model_generic, 4: v_drive_model_generic}
logprob_model_generic = models.loglogit(V_model_generic, av, travel_mode)
biogeme_model_generic = bio.BIOGEME(database, logprob_model_generic)
biogeme_model_generic.modelName = 'Model_generic'
all_results['Model_generic'] = biogeme_model_generic.estimate()
results_model_generic = biogeme_model_generic.estimate()
results_model_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.698422,1.213660,-2.223376,2.619047e-02
asc_cycle_male,1.056072,0.209817,5.033312,4.820785e-07
asc_drive,0.707880,0.899774,0.786731,4.314391e-01
asc_drive_male,-0.224704,0.101182,-2.220782,2.636573e-02
asc_pt,5.573208,1.725705,3.229526,1.239958e-03
asc_pt_male,-0.369760,0.108178,-3.418072,6.306631e-04
beta_cost,-0.170998,0.015858,-10.783335,0.000000e+00
beta_elapsed_cycle,-5.516379,2.115851,-2.607168,9.129461e-03
beta_elapsed_time_drive,-5.425926,2.415002,-2.246759,2.465546e-02
beta_elapsed_time_pt,-2.451080,1.035965,-2.365988,1.798201e-02


In [223]:
res.compileEstimationResults(all_results)

The syntax "compileEstimationResults" is deprecated and is replaced by the syntax "compile_estimation_results".


(                                              Model0           Model1  \
 Number of estimated parameters                     5                8   
 Sample size                                     5000             5000   
 Final log likelihood                    -4587.818071     -4275.496542   
 Akaike Information Criterion             9185.636142      8566.993084   
 Bayesian Information Criterion           9218.222108       8619.13063   
 asc_cycle (t-test)                      -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                     -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                       -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)                   -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                     -5.5  (-26.3)                    
 beta_time_cycle (t-test)                               -5.31  (-11.5)   
 beta_time_drive (t-test)                               -6.47  (-17.1)   
 beta_time_pt (t-test)                

In [224]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=191.55172530098207, threshold=11.070497693516351)

In [225]:
model_base=biogeme_model3
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=15.563995550181062, threshold=9.487729036781154)

## Model 4

We chose to separate the modes by nesting walking and cycling as "soft modes" and keeping driving and PT separeted. 

In [226]:
model_base=biogeme_model3
V_nested=V_model3
#create the nests
Mu_Drive =Beta("Mu_Drive",1,0,None,0)
Mu_PT =Beta("Mu_PT",1,0,None,0)
Mu_SM =Beta("Mu_SoftModes",1,0,None,0)
drive= Mu_Drive,[4]
PT=Mu_PT, [3]
SM=Mu_SM, [1,2]
nests_modes=drive, PT, SM

In [227]:
logprob_model_nested = models.lognested(V_nested, av, nests_modes, travel_mode)
biogeme_model_nested = bio.BIOGEME(database, logprob_model_nested)
biogeme_model_nested.modelName = 'nestmodes'
all_results['Nestmodes'] = biogeme_model_nested.estimate()
results_model_nested = biogeme_model_nested.estimate()
results_model_nested.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_Drive,1.000000,1.717057e-15,5.823918e+14,0.000000e+00
Mu_PT,1.000000,1.797693e+308,5.562685e-309,1.000000e+00
Mu_SoftModes,0.603680,7.647243e-02,7.894089e+00,2.886580e-15
asc_cycle,-2.955126,3.748133e-01,-7.884260e+00,3.108624e-15
asc_cycle_male,1.574252,3.494973e-01,4.504333e+00,6.658163e-06
asc_drive,0.294558,2.368810e-01,1.243484e+00,2.136895e-01
asc_drive_male,-0.147821,1.109199e-01,-1.332681e+00,1.826365e-01
asc_pt,2.181106,1.969323e-01,1.107541e+01,0.000000e+00
asc_pt_male,-0.290546,1.165381e-01,-2.493145e+00,1.266173e-02
beta_cost,-0.171546,1.569916e-02,-1.092708e+01,0.000000e+00


The model is rejected as Mu soft modes in < 1

We try another model : vehicles (bikes, cars, PT) and not vehicle (walk) 

In [228]:
Mu_vehicle=Beta("Mu_vehicle",1,0,None,0)
Mu_not_vehicle=Beta("Mu_not_vehicle",1,0,None,0)
vehicle= Mu_vehicle,[2,3,4]
not_vehicle=Mu_not_vehicle,[1]
nests_vehicle=vehicle, not_vehicle


In [229]:
logprob_model_nestedvehicle = models.lognested(V_nested, av, nests_vehicle, travel_mode)
biogeme_model_nestedvehicle = bio.BIOGEME(database, logprob_model_nestedvehicle)
biogeme_model_nestedvehicle.modelName = 'Model_nested_vehicle'
results_model_nestedvehicle = biogeme_model_nestedvehicle.estimate()
all_results['Model_nested_vehicle'] = results_model_nestedvehicle
table_nestedvehicle=results_model_nestedvehicle.getEstimatedParameters()
table_nestedvehicle

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_not_vehicle,1.000000,1.797693e+308,5.562685e-309,1.000000e+00
Mu_vehicle,1.141220,1.709776e-01,6.674674e+00,2.477840e-11
asc_cycle,-2.354804,9.594116e-01,-2.454425e+00,1.411100e-02
asc_cycle_male,0.900330,2.488584e-01,3.617841e+00,2.970712e-04
asc_drive,0.520332,5.210454e-01,9.986313e-01,3.179733e-01
asc_drive_male,-0.221890,1.000822e-01,-2.217078e+00,2.661776e-02
asc_pt,2.075767,2.938127e-01,7.064933e+00,1.606937e-12
asc_pt_male,-0.352541,1.071163e-01,-3.291199e+00,9.976138e-04
beta_cost,-0.153371,2.784415e-02,-5.508211e+00,3.624985e-08
beta_elapsed_cycle,-2.882741,5.249754e-01,-5.491192e+00,3.992300e-08


In [230]:
res.compileEstimationResults(all_results)

The syntax "compileEstimationResults" is deprecated and is replaced by the syntax "compile_estimation_results".


(                                              Model0           Model1  \
 Number of estimated parameters                     5                8   
 Sample size                                     5000             5000   
 Final log likelihood                    -4587.818071     -4275.496542   
 Akaike Information Criterion             9185.636142      8566.993084   
 Bayesian Information Criterion           9218.222108       8619.13063   
 asc_cycle (t-test)                      -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                     -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                       -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)                   -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                     -5.5  (-26.3)                    
 beta_time_cycle (t-test)                               -5.31  (-11.5)   
 beta_time_drive (t-test)                               -6.47  (-17.1)   
 beta_time_pt (t-test)                

In [231]:
results_base=model_base.estimate()
results_model_nestedvehicle.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 cannot be rejected at level 5.0%', statistic=0.9045581212012621, threshold=5.991464547107979)

So the model pref keeps being Model_3 (Box-cox transformation)

## Market Shares

The census data report the following number $N_g$ of individuals in each segment $g$.

In [232]:
census = {
    'male_45_more':  1379198,
    'male_45_less':  2926408,
    'female_45_more':   1519948,
    'female_45_less':  2841376,
}

The total size $N$ of the population is the sum over all segments.

In [233]:
total = sum(census.values())
total

8666930

Identify each segment in the database.

In [234]:
filters = {
    'male_45_more': (LPMC.age >=45) & (LPMC.female == 0),
    'male_45_less': (LPMC.age < 45) & (LPMC.female == 0),
    'female_45_more': (LPMC.age >= 45) & (LPMC.female == 1),
    'female_45_less': (LPMC.age < 45) & (LPMC.female == 1),
}

We count the number 
$S_g$ of individuals in each segment $g$ in the sample.

In [235]:

sample_segments = {
    k: v.sum() for k, v in filters.items()
}
sample_segments

{'male_45_more': 903,
 'male_45_less': 1436,
 'female_45_more': 997,
 'female_45_less': 1664}

We check the total sample size $S$.

In [236]:
total_sample = sum(sample_segments.values())
total_sample

5000

The weight $w_g$ associated with segment $g$ is defined as
$$
w_g = \frac{N_g}{N}\frac{S}{S_g}.
$$

In [237]:
weights = {
    k: census[k] * total_sample / (v * total) 
    for k, v in sample_segments.items()
}
weights

{'male_45_more': 0.881137295471826,
 'male_45_less': 1.17566922738916,
 'female_45_more': 0.8795049485193039,
 'female_45_less': 0.9850995060002171}

We insert the weight as a new column in the database.

In [238]:
for k, f in filters.items():
    LPMC.loc[f, 'Weight'] = weights[k] 

We check that the sum of the weights is the sample size.

In [239]:
sum_weights = LPMC['Weight'].sum()
sum_weights

5000.0

It is equal.

#### Perform sample enumeration

The goal is to estimate predicted shares using the formula :
    $$Ŵ(i) = \frac{1}{S}\sum_{n=1}^{S}w_n*P(i|x_n; θ)$$

First we need to obtain $P(i|x_n; θ)$ for each option $i$

Using Model 3 ( the preferred model so far ) and the logit model $$ P(i|x_n; θ)= \frac{exp(V_{in})}{\sum_{j \in C}{exp(V_{jn})}}$$ where $\mu$ is normalized to 1.

In [240]:
prob_walk =  exp(v_walk_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_drive = exp(v_drive_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_cycle = exp(v_cycle_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_pt =  exp(v_pt_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_model3)+exp(v_cycle_model3))

We calculate the choice probability of each alternative for each observation. We include the weight in the simulation results.

In [241]:
Weight = Variable('Weight')

In [242]:
simulate = {
    'Weight': Weight,
    'Prob. walk': prob_walk,
    'Prob. drive': prob_drive,
    'Prob. cycle': prob_cycle,
    'Prob. pt': prob_pt,    
}

In [243]:
database = db.Database('LPMC',  LPMC)
biosim = bio.BIOGEME(database, simulate)
simulated_values = biosim.simulate(results_model3.getBetaValues())

In [244]:
simulated_values


,Weight,Prob. walk,Prob. drive,Prob. cycle,Prob. pt
0,0.879505,0.001361,0.231612,0.010644,0.756383
1,0.881137,0.000098,0.199560,0.020164,0.780178
2,1.175669,0.029810,0.441930,0.037850,0.490409
3,1.175669,0.586454,0.203987,0.027478,0.182082
4,0.879505,0.413058,0.343854,0.017955,0.225134
...,...,...,...,...,...
4995,1.175669,0.449571,0.387903,0.044240,0.118286
4996,1.175669,0.000237,0.859190,0.010705,0.129868
4997,1.175669,0.105457,0.717432,0.063042,0.114069
4998,0.879505,0.123393,0.532410,0.026802,0.317396


Market shares are calculated using the weighted mean of the
individual probabilities.

In [245]:
simulated_values['Weighted walk'] = (
    simulated_values['Weight'] * 
    simulated_values['Prob. walk']
)
simulated_values['Weighted drive'] = (
    simulated_values['Weight'] * 
    simulated_values['Prob. drive']
)
simulated_values['Weighted cycle'] = (
    simulated_values['Weight'] * 
    simulated_values['Prob. cycle']
)
simulated_values['Weighted pt'] = (
    simulated_values['Weight'] * 
    simulated_values['Prob. pt']
)

In [246]:
market_share_walk = simulated_values['Weighted walk'].mean()
print(f'Market share for walk: {100*market_share_walk:.1f}%')
market_share_drive = simulated_values['Weighted drive'].mean()
print(f'Market share for drive: {100*market_share_drive:.1f}%')
market_share_cycle = simulated_values['Weighted cycle'].mean()
print(f'Market share for cycle: {100*market_share_cycle:.1f}%')
market_share_pt = simulated_values['Weighted pt'].mean()
print(f'Market share for pt: {100*market_share_pt:.1f}%')

print(market_share_walk)
print(market_share_drive)



Market share for walk: 17.8%
Market share for drive: 43.0%
Market share for cycle: 3.0%
Market share for pt: 36.2%
0.17802347584028297
0.42950515854071875


#### Perform sample enumeration

In order to calculate confidence intervals, we need to re-estimate the parameters using bootstrapping.

In [247]:
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.bootstrap_samples=100
results_bootstrapping = biogeme_model3.estimate(run_bootstrap=True)

You have not defined a name for the model. The output files are named from the model name. The default is [biogemeModelDefaultName]
 71%|█████████████████████████████████████████████████████████▌                       | 71/100 [01:27<00:35,  1.23s/it]


PermissionError: [Errno 13] Permission denied: '__biogemeModelDefaultName.iter'

We obtain a sample of values for the parameters. We use then to calculate empirically (that is, using simulation), the 90% confidence intervals on the simulated quantities.

In [ ]:
betas = biogeme_model3.freeBetaNames()
b = results_bootstrapping.getBetasForSensitivityAnalysis(betas, useBootstrap = False)

In [ ]:
left, right = biosim.confidenceIntervals(b, 0.9)

In [ ]:
left['Weighted walk'] = (
    left['Weight'] * 
    left['Prob. walk']
)
left['Weighted drive'] = (
    left['Weight'] * 
    left['Prob. drive']
)
left['Weighted pt'] = (
    left['Weight'] * 
    left['Prob. pt']
)
left['Weighted cycle'] = (
    left['Weight'] * 
    left['Prob. cycle']
)

In [ ]:
left_market_share_walk = left['Weighted walk'].mean()
left_market_share_drive = left['Weighted drive'].mean()
left_market_share_pt = left['Weighted pt'].mean()
left_market_share_cycle = left['Weighted cycle'].mean()

In [ ]:
right['Weighted walk'] = (
    right['Weight'] * 
    right['Prob. walk']
)
right['Weighted drive'] = (
    right['Weight'] * 
    right['Prob. drive']
)
right['Weighted pt'] = (
    right['Weight'] * 
    right['Prob. pt']
)
right['Weighted cycle'] = (
    right['Weight'] * 
    right['Prob. cycle']
)

In [ ]:
right_market_share_walk = right['Weighted walk'].mean()
right_market_share_drive = right['Weighted drive'].mean()
right_market_share_pt = right['Weighted pt'].mean()
right_market_share_cycle = right['Weighted cycle'].mean()


In [ ]:
print(
    f'Market share for walking: {100*market_share_walk:.1f}% '
    f'CI: ['
    f'{100*left_market_share_walk:.1f}%-'
    f'{100*right_market_share_walk:.1f}'
    f']'
)
print(
    f'Market share for driving: {100*market_share_drive:.1f}% '
    f'CI: ['
    f'{100*left_market_share_drive:.1f}%-'
    f'{100*right_market_share_drive:.1f}'
    f']'
)
print(
    f'Market share for public transportation: {100*market_share_pt:.1f}% '
    f'CI: ['
    f'{100*left_market_share_pt:.1f}%-'
    f'{100*right_market_share_pt:.1f}'
    f']'
)
print(
    f'Market share for cycling: {100*market_share_cycle:.1f}% '
    f'CI: ['
    f'{100*left_market_share_cycle:.1f}%-'
    f'{100*right_market_share_cycle:.1f}'
    f']'
)

#### Comparing with the weighted market shares computed using the actual choices

Instead of using the predictive Model we will use the actual choice of the individual from the sample and the weights computed earlier to calculate the Markets shares.

In [ ]:
LPMC['walk']= (LPMC.travel_mode==1)
LPMC['drive']= (LPMC.travel_mode==4)
LPMC['cycle']=(LPMC.travel_mode==2)
LPMC['pt']=(LPMC.travel_mode==3)

In [ ]:
LPMC ['Weighted walk real'] = (
    LPMC ['Weight'] * 
    LPMC ['walk']
)
LPMC ['Weighted drive real'] = (
    LPMC ['Weight'] * 
    LPMC ['drive']
)
LPMC ['Weighted cycle real'] = (
    LPMC ['Weight'] * 
    LPMC ['cycle']
)
LPMC ['Weighted pt real'] = (
    LPMC ['Weight'] * 
    LPMC ['pt']
)

In [ ]:
market_share_walk_real = (LPMC['Weighted walk real'].sum())/5000
print(f'Market share for walking from the sample: {100*market_share_walk_real:.1f}%')

market_share_cycle_real = (LPMC['Weighted cycle real'].sum())/5000
print(f'Market share for cycling from the sample: {100*market_share_cycle_real:.1f}%')

market_share_drive_real = (LPMC['Weighted drive real'].sum())/5000
print(f'Market share for driving from the sample: {100*market_share_drive_real:.1f}%')

market_share_pt_real = (LPMC['Weighted pt real'].sum())/5000
print(f'Market share for public transport from the sample: {100*market_share_pt_real:.1f}%')

## Quick Data Cleaning

We need to exlude the boolean variables created during the caluclation of market share because Jupyter can't simulate bolean

In [ ]:
my_cols = list(LPMC.columns)
 
# removing the desired column
my_cols.remove('walk')
my_cols.remove('drive')
my_cols.remove('cycle')
my_cols.remove('pt')



LPMC2 = LPMC[my_cols]

# Forecasting

### Scenario 1

Taking Model pref ( model 3) implemantation of the first scenario, where the price for automobilist increase by 1.5£ 

In [ ]:

v_drive_scenario1  = segmented_asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * (cost_drive+1.5)


Using the same method as the one we used to compute the market share before, we update the probability : $P(i|x_n; θ)$ for each option $i$ 

In [ ]:
prob_walk_s1 =  exp(v_walk_model3) /( exp(v_walk_model3)+ exp(v_drive_scenario1)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_drive_s1 = exp(v_drive_scenario1) /( exp(v_walk_model3)+ exp(v_drive_scenario1)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_cycle_s1 = exp(v_cycle_model3) /( exp(v_walk_model3)+ exp(v_drive_scenario1)+exp(v_pt_model3)+exp(v_cycle_model3))
prob_pt_s1 =  exp(v_pt_model3) /( exp(v_walk_model3)+ exp(v_drive_scenario1)+exp(v_pt_model3)+exp(v_cycle_model3))

Simulating the results

In [ ]:
simulate_1 = {
    'Weight': Weight,
    'Prob. walk s1': prob_walk_s1,
    'Prob. drive s1': prob_drive_s1,
    'Prob. cycle s1': prob_cycle_s1,
    'Prob. pt s1': prob_pt_s1,    
}


In [ ]:
database = db.Database('LPMC2',  LPMC2)
biosim1 = bio.BIOGEME(database, simulate_1)
simulated_values_s1 = biosim1.simulate(results_model3.getBetaValues())
simulated_values_s1


Observing the simulated data, we can see a slight decrease in Probability for the drive option

In [ ]:


simulated_values_s1['Weighted walk s1'] = (
    simulated_values_s1['Weight'] * 
    simulated_values_s1['Prob. walk s1']
)
simulated_values_s1['Weighted drive s1'] = (
    simulated_values_s1['Weight'] * 
    simulated_values_s1['Prob. drive s1']
)
simulated_values_s1['Weighted cycle s1'] = (
    simulated_values_s1['Weight'] * 
    simulated_values_s1['Prob. cycle s1']
)
simulated_values_s1['Weighted pt s1'] = (
    simulated_values_s1['Weight'] * 
    simulated_values_s1['Prob. pt s1']
)



Using the weight as before we compute the weighted of each option, and print the market share

In [ ]:
market_share_walk_s1 = simulated_values_s1['Weighted walk s1'].mean()

print(f'Market share for walk: {100*market_share_walk_s1:.1f}%')
market_share_drive_s1 = simulated_values_s1['Weighted drive s1'].mean()
print(f'Market share for drive: {100*market_share_drive_s1:.1f}%')
market_share_cycle_s1 = simulated_values_s1['Weighted cycle s1'].mean()
print(f'Market share for cycle: {100*market_share_cycle_s1:.1f}%')
market_share_pt_s1 = simulated_values_s1['Weighted pt s1'].mean()
print(f'Market share for pt: {100*market_share_pt_s1:.1f}%')

### Scenario 2

Taking Model pref ( model 3) we create the first scenario, where the price for public transport decrease by 20% 

In [ ]:

v_pt_scenario2  = segmented_asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt*0.8


In [ ]:
prob_walk_s2 =  exp(v_walk_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_scenario2)+exp(v_cycle_model3))
prob_drive_s2 = exp(v_drive_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_scenario2)+exp(v_cycle_model3))
prob_cycle_s2 = exp(v_cycle_model3) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_scenario2)+exp(v_cycle_model3))
prob_pt_s2 =  exp(v_pt_scenario2) /( exp(v_walk_model3)+ exp(v_drive_model3)+exp(v_pt_scenario2)+exp(v_cycle_model3))

In [ ]:
simulate_2 = {
    'Weight': Weight,
    'Prob. walk s2': prob_walk_s2,
    'Prob. drive s2': prob_drive_s2,
    'Prob. cycle s2': prob_cycle_s2,
    'Prob. pt s2': prob_pt_s2,    
}


In [ ]:
database = db.Database('LPMC2',  LPMC2)
biosim2 = bio.BIOGEME(database, simulate_2)
simulated_values_s2 = biosim2.simulate(results_model3.getBetaValues())
simulated_values_s2



In [ ]:
simulated_values_s2['Weighted walk s2'] = (
    simulated_values_s2['Weight'] * 
    simulated_values_s2['Prob. walk s2']
)
simulated_values_s2['Weighted drive s2'] = (
    simulated_values_s2['Weight'] * 
    simulated_values_s2['Prob. drive s2']
)
simulated_values_s2['Weighted cycle s2'] = (
    simulated_values_s2['Weight'] * 
    simulated_values_s2['Prob. cycle s2']
)
simulated_values_s2['Weighted pt s2'] = (
    simulated_values_s2['Weight'] * 
    simulated_values_s2['Prob. pt s2']
)


In [ ]:
market_share_walk_s2 = simulated_values_s2['Weighted walk s2'].mean()


print(f'Market share for walk: {100*market_share_walk_s2:.1f}%')
market_share_drive_s2 = simulated_values_s2['Weighted drive s2'].mean()
print(f'Market share for drive: {100*market_share_drive_s2:.1f}%')
market_share_cycle_s2 = simulated_values_s2['Weighted cycle s2'].mean()
print(f'Market share for cycle: {100*market_share_cycle_s1:.1f}%')
market_share_pt_s2 = simulated_values_s2['Weighted pt s2'].mean()
print(f'Market share for pt: {100*market_share_pt_s2:.1f}%')

The change is quite small !

#### Quick intuition on impact 

In [ ]:
LPMC['cost_transit'].mean()

Personnal Remark : On average the cost for public transportation is 1.5£ so decreasing it by 20% decrease in average would be 0.157 as the $beta$ is equal to -0.170998 ie it will increase the value function of pt by 0.026846686
Wheras increasing the price of car by 1.5 decrease its car value fucntion by 0.256497 (much bigger impact)

**Question 3**

In [ ]:
Total_revenue_no_scenario = (simulated_values['Weighted pt'].mean() * LPMC["cost_transit"]).sum()

Total_revenue_scenario_1 = (simulated_values_s1['Weighted pt s1'].mean() * LPMC["cost_transit"]).sum()

Total_revenue_scenario_2 = (simulated_values_s1['Weighted pt s1'].mean() * LPMC["cost_transit"] * 0.8).sum()

In [ ]:
# Print the results
print(f"Total Revenue - No Scenario: {Total_revenue_no_scenario }")
print(f"Total Revenue - Scenario 1: {Total_revenue_scenario_1 }")
print(f"Total Revenue - Scenario 2: {Total_revenue_scenario_2 }")


**Question 4**

In [ ]:
vot_pt = (Derive(v_pt_model3,'dur_pt') / Derive(v_pt_model3, 'cost_transit') )

In [ ]:
vot_drive = (Derive(v_drive_model3 ,'dur_pt') / Derive(v_drive_model3 , 'cost_transit') )

In [ ]:
simulate = {
    'Weight': Weight,
    'Prob. walk': prob_walk,
    'Prob. drive': prob_drive,
    'Prob. cycle': prob_cycle,
    'Prob. pt': prob_pt,    
    'VOT PT': vot_pt,
    'VOT CAR': vot_drive,
}
biosim = bio.BIOGEME(database, simulate)
simulated_vot_values = biosim.simulate(results_model3.getBetaValues())

In [ ]:
simulated_vot_values.loc[simulated_vot_values.cost_transit == 0, 'VOT PT'] = 0
simulated_vot_values.loc[simulated_vot_values.cost_driving_ccharge == 0, 'VOT CAR'] = 0
simulated_vot_values.loc[simulated_vot_values.cost_driving_fuel == 0, 'VOT CAR'] = 0

In [ ]:

_ = simulated_vot_values[simulated_vot_values['VOT CAR'] != 0]['VOT CAR'].hist(bins=200)

In [ ]:

_ = simulated_vot_values[simulated_vot_values['VOT PT'] != 0]['VOT PT'].hist(bins=200)

In [ ]:
avg_vot_pt = (
    simulated_vot_values['VOT PT'] * 
    simulated_vot_values['weights']
).sum() / simulated_vot_values['weights'].sum()
print(f'Average value of time for PT: {avg_vot_pt:.3g} CHF/hour')

In [ ]:
avg_vot_car = (
    simulated_vot_values['VOT CAR'] * 
    simulated_vot_values['weights']
).sum() / simulated_vot_values['weights'].sum()
print(f'Average value of time for car: {avg_vot_car:.3g} CHF/hour')

**Question 5**

In [249]:
 #Choice proba:
prob_WALK = models.nested(V_pref, av, nest_pref, 1)
prob_CYCLE = models.nested(V_pref, av, nest_pref, 2)
prob_PT = models.nested(V_pref, av, nest_pref, 3)
prob_DRIVE = models.nested(V_pref, av, nest_pref, 4)

NameError: name 'V_pref' is not defined